<center>
    <img src="https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/Logos/organization_logo/organization_logo.png" width="300" alt="cognitiveclass.ai logo"  />
</center>

<h1 align="center"><font size="5">Classification with Python</font></h1>


In this notebook we try to practice all the classification algorithms that we learned in this course.

We load a dataset using Pandas library, and apply the following algorithms, and find the best one for this specific dataset by accuracy evaluation methods.

Lets first load required libraries:


In [ ]:
import itertools
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
import pandas as pd
import numpy as np
import matplotlib.ticker as ticker
from sklearn import preprocessing
%matplotlib inline

### About dataset


This dataset is about past loans. The **Loan_train.csv** data set includes details of 346 customers whose loan are already paid off or defaulted. It includes following fields:

| Field          | Description                                                                           |
| -------------- | ------------------------------------------------------------------------------------- |
| Loan_status    | Whether a loan is paid off on in collection                                           |
| Principal      | Basic principal loan amount at the                                                    |
| Terms          | Origination terms which can be weekly (7 days), biweekly, and monthly payoff schedule |
| Effective_date | When the loan got originated and took effects                                         |
| Due_date       | Since it’s one-time payoff schedule, each loan has one single due date                |
| Age            | Age of applicant                                                                      |
| Education      | Education of applicant                                                                |
| Gender         | The gender of applicant                                                               |


Lets download the dataset


In [5]:
!wget -O loan_train.csv https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML0101EN-SkillsNetwork/labs/FinalModule_Coursera/data/loan_train.csv

--2020-11-19 19:43:48--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML0101EN-SkillsNetwork/labs/FinalModule_Coursera/data/loan_train.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 67.228.254.196
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23101 (23K) [text/csv]
Saving to: ‘loan_train.csv’

loan_train.csv      100%[===================>]  22.56K  --.-KB/s    in 0.07s   

2020-11-19 19:43:49 (304 KB/s) - ‘loan_train.csv’ saved [23101/23101]



### Load Data From CSV File


In [1]:
df = pd.read_csv('loan_train.csv')
df.head()

NameError: name 'pd' is not defined

In [ ]:
df.shape

### Convert to date time object


In [26]:
df['due_date'] = pd.to_datetime(df['due_date'])
df['effective_date'] = pd.to_datetime(df['effective_date'])
df.head()

,Unnamed: 0,Unnamed: 0.1,loan_status,Principal,terms,effective_date,due_date,age,education,Gender
0,0,0,PAIDOFF,1000,30,2016-09-08,2016-10-07,45,High School or Below,male
1,2,2,PAIDOFF,1000,30,2016-09-08,2016-10-07,33,Bechalor,female
2,3,3,PAIDOFF,1000,15,2016-09-08,2016-09-22,27,college,male
3,4,4,PAIDOFF,1000,30,2016-09-09,2016-10-08,28,college,female
4,6,6,PAIDOFF,1000,30,2016-09-09,2016-10-08,29,college,male


# Data visualization and pre-processing


Let’s see how many of each class is in our data set 


In [ ]:
df['loan_status'].value_counts()

260 people have paid off the loan on time while 86 have gone into collection 


Lets plot some columns to underestand data better:


In [ ]:
# notice: installing seaborn might takes a few minutes
!conda install -c anaconda seaborn -y

In [ ]:
import seaborn as sns

bins = np.linspace(df.Principal.min(), df.Principal.max(), 10)
g = sns.FacetGrid(df, col="Gender", hue="loan_status", palette="Set1", col_wrap=2)
g.map(plt.hist, 'Principal', bins=bins, ec="k")

g.axes[-1].legend()
plt.show()

In [ ]:
bins = np.linspace(df.age.min(), df.age.max(), 10)
g = sns.FacetGrid(df, col="Gender", hue="loan_status", palette="Set1", col_wrap=2)
g.map(plt.hist, 'age', bins=bins, ec="k")

g.axes[-1].legend()
plt.show()

# Pre-processing:  Feature selection/extraction


### Lets look at the day of the week people get the loan


In [ ]:
df['dayofweek'] = df['effective_date'].dt.dayofweek
bins = np.linspace(df.dayofweek.min(), df.dayofweek.max(), 10)
g = sns.FacetGrid(df, col="Gender", hue="loan_status", palette="Set1", col_wrap=2)
g.map(plt.hist, 'dayofweek', bins=bins, ec="k")
g.axes[-1].legend()
plt.show()


We see that people who get the loan at the end of the week dont pay it off, so lets use Feature binarization to set a threshold values less then day 4 


In [ ]:
df['weekend'] = df['dayofweek'].apply(lambda x: 1 if (x>3)  else 0)
df.head()

## Convert Categorical features to numerical values


Lets look at gender:


In [ ]:
df.groupby(['Gender'])['loan_status'].value_counts(normalize=True)

86 % of female pay there loans while only 73 % of males pay there loan


Lets convert male to 0 and female to 1:


In [ ]:
df['Gender'].replace(to_replace=['male','female'], value=[0,1],inplace=True)
df.head()

## One Hot Encoding

#### How about education?


In [ ]:
df.groupby(['education'])['loan_status'].value_counts(normalize=True)

#### Feature befor One Hot Encoding


In [ ]:
df[['Principal','terms','age','Gender','education']].head()

#### Use one hot encoding technique to conver categorical varables to binary variables and append them to the feature Data Frame


In [ ]:
Feature = df[['Principal','terms','age','Gender','weekend']]
Feature = pd.concat([Feature,pd.get_dummies(df['education'])], axis=1)
Feature.drop(['Master or Above'], axis = 1,inplace=True)
Feature.head()


### Feature selection


Lets defind feature sets, X:


In [ ]:
X = Feature
X[0:5]

What are our lables?


In [ ]:
y = df['loan_status'].values
y[0:5]

## Normalize Data


Data Standardization give data zero mean and unit variance (technically should be done after train test split )


In [ ]:
X= preprocessing.StandardScaler().fit(X).transform(X)
X[0:5]

# Classification


Now, it is your turn, use the training set to build an accurate model. Then use the test set to report the accuracy of the model
You should use the following algorithm:

-   K Nearest Neighbor(KNN)
-   Decision Tree
-   Support Vector Machine
-   Logistic Regression

** Notice:** 

-   You can go above and change the pre-processing, feature selection, feature-extraction, and so on, to make a better model.
-   You should use either scikit-learn, Scipy or Numpy libraries for developing the classification algorithms.
-   You should include the code of the algorithm in the following cells.


# K Nearest Neighbor(KNN)

Notice: You should find the best k to build the model with the best accuracy.  
**warning:** You should not use the **loan_test.csv** for finding the best k, however, you can split your train_loan.csv into train and test to find the best **k**.


In [6]:
!wget -O loan_train.csv https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML0101EN-SkillsNetwork/labs/FinalModule_Coursera/data/loan_train.csv
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

#loading the data set
df = pd.read_csv('loan_train.csv')


#finding the best k
besty = len(df.index)
print(besty)




le_gender = LabelEncoder()
le_education = LabelEncoder()
f=df.drop(['Unnamed: 0','Unnamed: 0.1','effective_date','due_date'],axis='columns')
f['educationN'] = le_education.fit_transform(f['education'])
f['genderN'] = le_gender.fit_transform(f['Gender'])
f = f.drop(['education','Gender'],axis='columns')


#X are the attributes like principal etc
X = f.iloc[:,1:7]


#Y are the labels in this case the loan_status
Y = f.iloc[:,0]

classifier = KNeighborsClassifier(n_neighbors=18)
classifier.fit(X,Y)








--2020-11-23 11:44:41--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML0101EN-SkillsNetwork/labs/FinalModule_Coursera/data/loan_train.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 67.228.254.196
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23101 (23K) [text/csv]
Saving to: ‘loan_train.csv’

loan_train.csv      100%[===================>]  22.56K  --.-KB/s    in 0.07s   

2020-11-23 11:44:42 (306 KB/s) - ‘loan_train.csv’ saved [23101/23101]

346


KNeighborsClassifier(n_neighbors=18)

# Decision Tree


In [5]:
!wget -O loan_train.csv https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML0101EN-SkillsNetwork/labs/FinalModule_Coursera/data/loan_train.csv

--2020-11-21 16:35:32--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML0101EN-SkillsNetwork/labs/FinalModule_Coursera/data/loan_train.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 67.228.254.196
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23101 (23K) [text/csv]
Saving to: ‘loan_train.csv’

loan_train.csv      100%[===================>]  22.56K  --.-KB/s    in 0.07s   

2020-11-21 16:35:33 (305 KB/s) - ‘loan_train.csv’ saved [23101/23101]



In [7]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn import tree


In [8]:
le_gender = LabelEncoder()
le_education = LabelEncoder()
le_status = LabelEncoder()

df = pd.read_csv('loan_train.csv')

target = df['loan_status'] 


final_file = df.drop(['loan_status','Unnamed: 0','Unnamed: 0.1','effective_date','due_date'],axis='columns')
final_file.head()

final_file['educationN'] = le_education.fit_transform(final_file['education'])
final_file['genderN'] = le_education.fit_transform(final_file['Gender'])

vb = final_file.drop(['education','Gender'],axis='columns')
print(vb.head)
model = tree.DecisionTreeClassifier()
model.fit(vb[['Principal','terms','age','educationN','genderN']],target)

<bound method NDFrame.head of      Principal  terms  age  educationN  genderN
0         1000     30   45           1        1
1         1000     30   33           0        0
2         1000     15   27           3        1
3         1000     30   28           3        0
4         1000     30   29           3        1
..         ...    ...  ...         ...      ...
341        800     15   32           1        1
342       1000     30   25           1        1
343        800     15   39           3        1
344       1000     30   28           3        1
345       1000     30   26           3        1

[346 rows x 5 columns]>


DecisionTreeClassifier()

# Support Vector Machine


In [3]:
!wget -O loan_train.csv https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML0101EN-SkillsNetwork/labs/FinalModule_Coursera/data/loan_train.csv

--2020-11-21 12:47:17--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML0101EN-SkillsNetwork/labs/FinalModule_Coursera/data/loan_train.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 67.228.254.196
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23101 (23K) [text/csv]
Saving to: ‘loan_train.csv’

loan_train.csv      100%[===================>]  22.56K  --.-KB/s    in 0.07s   

2020-11-21 12:47:17 (304 KB/s) - ‘loan_train.csv’ saved [23101/23101]



In [9]:
from sklearn.svm import SVC
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder
import pandas as pd

file = pd.read_csv('loan_train.csv')

final = file.drop(['Unnamed: 0','Unnamed: 0.1','loan_status','effective_date','due_date'],axis='columns')
target = file['loan_status']

le_education = LabelEncoder()
le_gender = LabelEncoder()
final.head()
final['educationN'] = le_education.fit_transform(final['education'])
final['genderN'] = le_gender.fit_transform(final['Gender'])

data = final.drop(['education','Gender'],axis='columns')
data.head()

,Principal,terms,age,educationN,genderN
0,1000,30,45,1,1
1,1000,30,33,0,0
2,1000,15,27,3,1
3,1000,30,28,3,0
4,1000,30,29,3,1


In [10]:
m = SVC(gamma='auto')
m.fit(data[['Principal','terms','age','educationN','genderN']],target)
print()

# Logistic Regression


In [2]:
!wget -O loan_train.csv https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML0101EN-SkillsNetwork/labs/FinalModule_Coursera/data/loan_train.csv

--2020-11-21 09:47:44--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML0101EN-SkillsNetwork/labs/FinalModule_Coursera/data/loan_train.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 67.228.254.196
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23101 (23K) [text/csv]
Saving to: ‘loan_train.csv’

loan_train.csv      100%[===================>]  22.56K  --.-KB/s    in 0.07s   

2020-11-21 09:47:45 (304 KB/s) - ‘loan_train.csv’ saved [23101/23101]



In [11]:

import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
df = pd.read_csv('loan_train.csv')

le_gender = LabelEncoder()
le_education = LabelEncoder()

i = df.drop('Unnamed: 0',axis='columns')
j = i.drop('Unnamed: 0.1',axis='columns')

j['Gendern'] = le_gender.fit_transform(j['Gender'])
j['EducationN'] = le_gender.fit_transform(j['education'])

final_data = j.drop(['loan_status','education','Gender'],axis='columns')



In [12]:
train_x = final_data[['Principal','terms','age','Gendern','EducationN']]
y = df['loan_status']
model = LogisticRegression(max_iter=1000)
model.fit(final_data[['Principal','terms','age','Gendern','EducationN']],y)


LogisticRegression(max_iter=1000)

# Model Evaluation using Test set


In [13]:
from sklearn.metrics import jaccard_score
from sklearn.metrics import f1_score
from sklearn.metrics import log_loss
from sklearn.preprocessing import LabelEncoder


First, download and load the test set:


In [14]:
!wget -O loan_test.csv https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/loan_test.csv

--2020-11-23 11:45:07--  https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/loan_test.csv
Resolving s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)... 67.228.254.196
Connecting to s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3642 (3.6K) [text/csv]
Saving to: ‘loan_test.csv’

loan_test.csv       100%[===================>]   3.56K  --.-KB/s    in 0s      

2020-11-23 11:45:07 (61.9 MB/s) - ‘loan_test.csv’ saved [3642/3642]



### Load Test set for evaluation


In [15]:
import pandas as pd


#accuracy evaluation of KNN Algorithms
testing = pd.read_csv('loan_test.csv')

loans = LabelEncoder()
education = LabelEncoder()
gender = LabelEncoder()
pred = LabelEncoder()
testing['loan_N'] = loans.fit_transform(testing['loan_status'])
testing['education_N'] = education.fit_transform(testing['education'])
testing['gender_N'] = gender.fit_transform(testing['Gender'])

tart = testing['loan_N']

testing = testing.drop(['loan_status','effective_date','due_date','Gender','education','Unnamed: 0.1','Unnamed: 0','loan_N'],axis='columns')


testing.head()

predicted_values = classifier.predict(testing)
pred_val = pred.fit_transform(predicted_values)


f1_score_knn = f1_score(tart,pred_val,average='weighted')




jaccard_score_knn = jaccard_score(tart,pred_val,average=None)



log_loss_knn = log_loss(tart,pred_val)



                 



In [16]:
#evaluation for decision tree classifier


pd = LabelEncoder()
pred_dec = model.predict(testing)
pred_dec = pd.fit_transform(pred_dec)

f1_score_decision = f1_score(tart,pred_dec,average='weighted')
jaccard_score_decision = jaccard_score(tart,pred_dec,average=None)
log_loss_decision = log_loss(tart,pred_dec)




In [17]:
#evaluation for support vector machine
sv_encod = LabelEncoder()
svr = m.predict(testing)
svr = sv_encod.fit_transform(svr)
f1_score_SVM = f1_score(tart,svr,average='weighted')
jaccard_score_SVM = jaccard_score(tart,svr,average=None)
log_loss_SVM = log_loss(tart,svr)


In [27]:
#evaluation for logistic regression
log_encod = LabelEncoder()
log_pred = model.predict(testing)
log_pred = log_encod.fit_transform(log_pred)
f1_score_log = f1_score(tart,log_pred,average='weighted')
jaccard_score_log = jaccard_score(tart,log_pred,average=None)
log_loss_log = log_loss(tart,log_pred)




#printing the final result
print('Results for KNN are as follows: ')
print('jaccard: ',jaccard_score_knn)
print('f1-score: ',f1_score_knn)
print('logloss ',log_loss_knn)
print('\n')
print('Results for decision are as follows: ')
print('jaccard: ',jaccard_score_decision)
print('f1-score: ',f1_score_decision)
print('logloss ',log_loss_decision)
print('\n')
print('Results for SVM are as follows: ')
print('jaccard: ',jaccard_score_SVM)
print('f1-score: ',f1_score_SVM)
print('logloss ',log_loss_SVM)
print('\n')
print('Results for logistic are as follows: ')
print('jaccard: ',jaccard_score_log)
print('f1-score: ',f1_score_log)
print('logloss ',log_loss_log)
print('\n')


Results for KNN are as follows: 
jaccard:  [0.25925926 0.        ]
f1-score:  0.10675381263616558
logloss  25.58427881104495


Results for decision are as follows: 
jaccard:  [0.125 0.44 ]
f1-score:  0.5102880658436214
logloss  17.90914324132965


Results for SVM are as follows: 
jaccard:  [0.         0.72222222]
f1-score:  0.6212664277180406
logloss  9.594311857179314


Results for logistic are as follows: 
jaccard:  [0.125 0.44 ]
f1-score:  0.5102880658436214
logloss  17.90914324132965




# Report

You should be able to report the accuracy of the built model using different evaluation metrics:


| Algorithm          | Jaccard | F1-score | LogLoss |
| ------------------ | ------- | -------- | ------- |
| KNN                | ?       | ?        | NA      |
| Decision Tree      | ?       | ?        | NA      |
| SVM                | ?       | ?        | NA      |
| LogisticRegression | ?       | ?        | ?       |


<h2>Want to learn more?</h2>

IBM SPSS Modeler is a comprehensive analytics platform that has many machine learning algorithms. It has been designed to bring predictive intelligence to decisions made by individuals, by groups, by systems – by your enterprise as a whole. A free trial is available through this course, available here: <a href="http://cocl.us/ML0101EN-SPSSModeler">SPSS Modeler</a>

Also, you can use Watson Studio to run these notebooks faster with bigger datasets. Watson Studio is IBM's leading cloud solution for data scientists, built by data scientists. With Jupyter notebooks, RStudio, Apache Spark and popular libraries pre-packaged in the cloud, Watson Studio enables data scientists to collaborate on their projects without having to install anything. Join the fast-growing community of Watson Studio users today with a free account at <a href="https://cocl.us/ML0101EN_DSX">Watson Studio</a>

<h3>Thanks for completing this lesson!</h3>

<h4>Author:  <a href="https://ca.linkedin.com/in/saeedaghabozorgi">Saeed Aghabozorgi</a></h4>
<p><a href="https://ca.linkedin.com/in/saeedaghabozorgi">Saeed Aghabozorgi</a>, PhD is a Data Scientist in IBM with a track record of developing enterprise level applications that substantially increases clients’ ability to turn data into actionable knowledge. He is a researcher in data mining field and expert in developing advanced analytic methods like machine learning and statistical modelling on large datasets.</p>

<hr>

## Change Log

| Date (YYYY-MM-DD) | Version | Changed By    | Change Description                                                             |
| ----------------- | ------- | ------------- | ------------------------------------------------------------------------------ |
| 2020-10-27        | 2.1     | Lakshmi Holla | Made changes in import statement due to updates in version of  sklearn library |
| 2020-08-27        | 2.0     | Malika Singla | Added lab to GitLab                                                            |

<hr>

## <h3 align="center"> © IBM Corporation 2020. All rights reserved. <h3/>

<p>
